In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import torch
from torch import nn 
import torch.nn.functional as F
from torch.utils import data

torch.manual_seed(0)
np.random.seed(0)

In [3]:
# install skorch
#!pip install -U skorch

## Step 1: Read data and preprocessing

These are exactly the same as Task 2 except we do not split a validation set. This will be automatically taken care of by Skorch

In [4]:
# load dataset for binary classification
data = pd.read_csv('pima-indians-diabetes.csv', header= None)
data.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
# split data into X and y
X = data.iloc[:,0:8].values
y = data.iloc[:,8].values

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# normalization 
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# convert to torch object
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

In [6]:
# print out the size of train, val and test set
print('train size:', X_train.shape)
print('test size:', X_test.shape)

train size: torch.Size([576, 8])
test size: torch.Size([192, 8])


## Step 2: Define a Pytorch model class

First we define a model similar to the one in Task 2. The difference is, we are passing dropout probability as a parameter in the initialization of the model. This is for the purpose of tuning this parameter.

In [7]:
class NNClassifier(torch.nn.Module):
    def __init__(self, inputSize, hiddenSize, p):
        super().__init__() 
        
        self.Linear1 = nn.Linear(inputSize, hiddenSize)
        self.Linear2 = nn.Linear(hiddenSize, hiddenSize)
        self.Linear3 = nn.Linear(hiddenSize, 1)
        
        self.act1 = nn.ReLU()
        self.act2 = nn.ReLU()
        
        # add two dropout layers with probability p = 0.2
        # Your code here
        self.Dropout1 = nn.Dropout(p=p)
        self.Dropout2 = nn.Dropout(p=p)
        
        
    def forward(self, x):        
        # first layer (linear + act + dropout)
        # Your code here
        x = self.Linear1(x)
        x = self.act1(x)
        x = self.Dropout1(x)
        
        # second layer (linear + act + dropout)
        # Your code here
        x = self.Linear2(x)
        x = self.act2(x)
        x = self.Dropout2(x)
        
        # output layer (no activation)
        # Your code here
        x = self.Linear3(x)
        
        return x

## Step 3: Wrap this model with skorch module

After we define a model in Pytorch. We use `NeuralNetBinaryClassifier` to wrap our defined model and turn it into a model that is compatible with `sklearn`. For multiclass classification and regression, you may use `NeuralNetClassifier` and `NeuralNetRegressor` instead. Notice *You need to specify the inputs when wrapping your model.*

In addition, **early stopping** can be easily applied in skorch, which is implemented in `callbacks`. It offers a more flexible way to stop the training. Instead of stopping the training whenever the validation loss does not increase, it allows some tolerance. 

In [7]:
from skorch import NeuralNetBinaryClassifier
from skorch.callbacks import EarlyStopping

In [8]:
net = NeuralNetBinaryClassifier(
    module = NNClassifier,
    
    # == hyperparameters for model definition ==
    # Init arguments for your module should be passed to NeuralNet with the module__ prefix
    module__inputSize = X_train.shape[1],
    module__hiddenSize = 256,
    module__p = 0.1,
    
    # == hyperparameters loss and optimizers ==
    # Note some of these parameters are set by default, check the default values:
    # https://skorch.readthedocs.io/en/stable/classifier.html
    criterion=torch.nn.BCEWithLogitsLoss, 
    optimizer = torch.optim.SGD,
    optimizer__lr = 0.1,
    optimizer__weight_decay = 0.01,
    batch_size = 16,
    
    # == hyperparameters for step 3: training ==
    max_epochs = 1000,
    # new we specify early stopping in callbacks: we stop when valid_loss does not improve for 10 epochs
    # we may use validation acc as measure to monitor training progress
    callbacks = [
        ('EarlyStopping', EarlyStopping(monitor='valid_loss', patience=10))
    ]
    
)

In [9]:
net.fit(X_train, y_train.float())

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6596       0.6379        0.6551  0.0674
      2        0.6475       0.6379        0.6441  0.0452
      3        0.6358       0.6379        0.6316  0.0604
      4        0.6248       0.6379        0.6193  0.0519
      5        0.6135       0.6552        0.6051  0.0555
      6        0.5970       0.6810        0.5904  0.0400
      7        0.5822       0.6983        0.5751  0.0491
      8        0.5738       0.7241        0.5630  0.0520
      9        0.5587       0.7328        0.5529  0.0392
     10        0.5518       0.7328        0.5444  0.0645
     11        0.5388       0.7241        0.5376  0.0638
     12        0.5407       0.7241        0.5338  0.0580
     13        0.5297       0.7241        0.5286  0.0557
     14        0.5354       0.7241        0.5249  0.0526
     15        0.5286       0.7328        0.5225  0.0492
     16        0.5249       0.7

<class 'skorch.classifier.NeuralNetBinaryClassifier'>[initialized](
  module_=NNClassifier(
    (Linear1): Linear(in_features=8, out_features=256, bias=True)
    (Linear2): Linear(in_features=256, out_features=256, bias=True)
    (Linear3): Linear(in_features=256, out_features=1, bias=True)
    (act1): ReLU()
    (act2): ReLU()
    (Dropout1): Dropout(p=0.1, inplace=False)
    (Dropout2): Dropout(p=0.1, inplace=False)
  ),
)

Evaluate test performance.

In [10]:
# prediction for test data
y_pred = net.predict(X_test)
print(classification_report(y_test.squeeze(),y_pred))

              precision    recall  f1-score   support

           0       0.81      0.88      0.84       130
           1       0.69      0.58      0.63        62

    accuracy                           0.78       192
   macro avg       0.75      0.73      0.74       192
weighted avg       0.77      0.78      0.78       192



## Step 4: Hyperparameter tuning

In step 3, we have built a neural network model in skorch without tuning the hyperparameter. Now let's use `RandomizedSearchCV` to tune hyperparameters. It is the same as any other models in `sklearn`. 

Note we have already specified some default values for hyperparameters when we define the `net` model. Thus, if we don't tune some hyperparameters, they will be fixed to their default values.

In [11]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'module__hiddenSize': [ 8, 16, 64, 128 ],
    'module__p': [0, 0.1, 0.2],
    
    'optimizer__weight_decay': [0, 0.01, 0.1],
    'batch_size': [8, 16],
    
    
    'callbacks': [
                  [('EarlyStopping', EarlyStopping(monitor='valid_loss', patience=5))],    
                  [('EarlyStopping', EarlyStopping(monitor='valid_loss', patience=10))] 
                 ]
}

Next, we use `RandomizedSearchCV`. Note `n_iter = 30` means we only sample 30 combinations of hyperparameters rather than all combinations as in `GridSearchCV`. The following code will take some time to run.

In [12]:
%%time
rscv = RandomizedSearchCV(net, params, cv=5, n_iter = 30, scoring='accuracy', random_state=0)
rscv.fit(X_train, y_train.float())

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6739       0.6522        0.6504  0.0613
      2        0.6562       0.6522        0.6479  0.0527
      3        0.6546       0.6522        0.6462  0.0548
      4        0.6555       0.6522        0.6455  0.0529
      5        0.6550       0.6522        0.6446  0.0560
      6        0.6527       0.6522        0.6437  0.0452
      7        0.6517       0.6522        0.6433  0.0453
      8        0.6517       0.6522        0.6424  0.0470
      9        0.6512       0.6522        0.6416  0.0690
     10        0.6493       0.6522        0.6408  0.0770
     11        0.6491       0.6522        0.6392  0.0930
     12        0.6500       0.6522        0.6381  0.0475
     13        0.6482       0.6522        0.6366  0.0741
     14        0.6462       0.6522        0.6355  0.0760
     15        0.6444       0.6522        0.6346  0.0566
     16        0.6443       0.6

     10        0.6503       0.6452        0.6463  0.0432
     11        0.6469       0.6452        0.6450  0.0421
     12        0.6471       0.6452        0.6437  0.0418
     13        0.6472       0.6452        0.6431  0.0444
     14        0.6520       0.6452        0.6434  0.0366
     15        0.6450       0.6452        0.6413  0.0423
     16        0.6426       0.6452        0.6394  0.0369
     17        0.6414       0.6452        0.6371  0.0450
     18        0.6394       0.6452        0.6349  0.0436
     19        0.6398       0.6452        0.6347  0.0436
     20        0.6348       0.6452        0.6310  0.0421
     21        0.6363       0.6452        0.6273  0.0468
     22        0.6290       0.6452        0.6238  0.0468
     23        0.6223       0.6452        0.6239  0.0586
     24        0.6128       0.6667        0.6132  0.0629
     25        0.6267       0.6559        0.6184  0.0531
     26        0.6204       0.6667        0.6099  0.0464
     27        0.6093       0.6

     13        0.6434       0.6452        0.6413  0.0439
     14        0.6436       0.6452        0.6417  0.0448
     15        0.6393       0.6452        0.6387  0.0467
     16        0.6406       0.6452        0.6406  0.0377
     17        0.6429       0.6452        0.6382  0.0399
     18        0.6333       0.6452        0.6348  0.0397
     19        0.6320       0.6452        0.6352  0.0410
     20        0.6296       0.6452        0.6338  0.0460
     21        0.6201       0.6452        0.6252  0.0399
     22        0.6104       0.6452        0.6284  0.0501
     23        0.6212       0.6452        0.6268  0.0432
     24        0.6144       0.6452        0.6265  0.0386
     25        0.6129       0.6452        0.6233  0.0421
     26        0.6141       0.6452        0.6136  0.0400
     27        0.5990       0.6452        0.6325  0.0510
     28        0.5682       0.6774        0.5922  0.0411
     29        0.5959       0.6452        0.6537  0.0419
     30        0.5794       0.6

     28        0.5817       0.7174        0.5550  0.0334
     29        0.5779       0.7283        0.5472  0.0322
     30        0.5693       0.7609        0.5401  0.0315
     31        0.5675       0.7609        0.5341  0.0229
     32        0.5569       0.7717        0.5272  0.0267
     33        0.5640       0.7717        0.5210  0.0247
     34        0.5596       0.8043        0.5200  0.0339
     35        0.5655       0.8043        0.5157  0.0232
     36        0.5497       0.8043        0.5126  0.0348
     37        0.5389       0.7935        0.5033  0.0336
     38        0.5400       0.8043        0.5010  0.0293
     39        0.5493       0.7935        0.5015  0.0312
     40        0.5316       0.7826        0.5009  0.0269
     41        0.5280       0.7826        0.4989  0.0332
     42        0.5444       0.7826        0.4948  0.0320
     43        0.5290       0.7826        0.4955  0.0319
     44        0.5277       0.7935        0.4970  0.0388
     45        0.5319       0.7

      5        0.6461       0.6452        0.6402  0.0320
      6        0.6390       0.6452        0.6367  0.0425
      7        0.6349       0.6452        0.6320  0.0368
      8        0.6304       0.6452        0.6275  0.0423
      9        0.6253       0.6452        0.6226  0.0353
     10        0.6199       0.6452        0.6171  0.0357
     11        0.6092       0.6452        0.6114  0.0404
     12        0.6040       0.6559        0.6050  0.0282
     13        0.5988       0.6667        0.5986  0.0361
     14        0.5880       0.6882        0.5927  0.0358
     15        0.5802       0.7312        0.5860  0.0362
     16        0.5706       0.7312        0.5813  0.0344
     17        0.5714       0.7419        0.5776  0.0361
     18        0.5582       0.7204        0.5722  0.0316
     19        0.5575       0.7097        0.5697  0.0317
     20        0.5456       0.7097        0.5669  0.0297
     21        0.5507       0.7097        0.5652  0.0342
     22        0.5460       0.7

...

Stopping since valid_loss has not improved in the last 10 epochs.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6712       0.6452        0.6564  0.0357
      2        0.6595       0.6452        0.6545  0.0315
      3        0.6582       0.6452        0.6541  0.0370
      4        0.6574       0.6452        0.6541  0.0408
      5        0.6573       0.6452        0.6540  0.0282
      6        0.6573       0.6452        0.6539  0.0341
      7        0.6572       0.6452        0.6539  0.0293
      8        0.6570       0.6452        0.6538  0.0266
      9        0.6571       0.6452        0.6538  0.0302
     10        0.6570       0.6452        0.6538  0.0213
     11        0.6569       0.6452        0.6537  0.0224
     12        0.6569       0.6452        0.6537  0.0227
     13        0.6569       0.6452        0.6537  0.0247
     14        0.6569       0.6452        0.6537  0.0224
     15        0.6569 

      5        0.6371       0.6522        0.6126  0.0353
      6        0.6284       0.6522        0.5979  0.0376
      7        0.6172       0.6630        0.5798  0.0399
      8        0.6040       0.6848        0.5588  0.0344
      9        0.5900       0.7609        0.5376  0.0351
     10        0.5774       0.7609        0.5194  0.0438
     11        0.5665       0.7826        0.5052  0.0472
     12        0.5574       0.7935        0.4951  0.0414
     13        0.5500       0.7935        0.4886  0.0500
     14        0.5439       0.7935        0.4848  0.0303
     15        0.5384       0.7826        0.4815  0.0330
     16        0.5340       0.7935        0.4792  0.0434
     17        0.5294       0.7826        0.4769  0.0420
     18        0.5255       0.7826        0.4745  0.0417
     19        0.5217       0.7717        0.4732  0.0385
     20        0.5194       0.7717        0.4717  0.0415
     21        0.5161       0.7826        0.4690  0.0410
     22        0.5137       0.7

     31        0.4945       0.7204        0.5407  0.0351
     32        0.4936       0.7312        0.5405  0.0364
     33        0.4924       0.7204        0.5405  0.0459
     34        0.4908       0.7204        0.5404  0.0352
     35        0.4899       0.7204        0.5399  0.0351
     36        0.4890       0.7312        0.5392  0.0509
     37        0.4884       0.7204        0.5395  0.0387
     38        0.4875       0.7312        0.5389  0.0612
     39        0.4864       0.7312        0.5387  0.0372
     40        0.4852       0.7312        0.5385  0.0345
     41        0.4843       0.7312        0.5377  0.0292
     42        0.4834       0.7312        0.5369  0.0358
     43        0.4828       0.7312        0.5367  0.0374
     44        0.4824       0.7312        0.5369  0.0453
     45        0.4816       0.7204        0.5372  0.0387
     46        0.4808       0.7204        0.5359  0.0285
     47        0.4806       0.7204        0.5360  0.0354
     48        0.4798       0.7

     39        0.4521       0.7742        0.5115  0.0367
     40        0.4513       0.7742        0.5115  0.0370
     41        0.4505       0.7742        0.5116  0.0365
     42        0.4497       0.7742        0.5105  0.0323
     43        0.4489       0.7742        0.5118  0.0351
     44        0.4483       0.7742        0.5115  0.0431
     45        0.4472       0.7742        0.5112  0.0361
     46        0.4466       0.7742        0.5104  0.0397
     47        0.4459       0.7742        0.5108  0.0400
     48        0.4450       0.7742        0.5107  0.0530
     49        0.4445       0.7634        0.5102  0.0328
     50        0.4437       0.7634        0.5107  0.0446
     51        0.4431       0.7634        0.5106  0.0363
     52        0.4425       0.7634        0.5107  0.0386
     53        0.4420       0.7634        0.5111  0.0370
Stopping since valid_loss has not improved in the last 5 epochs.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  

RandomizedSearchCV(cv=5,
                   estimator=<class 'skorch.classifier.NeuralNetBinaryClassifier'>[initialized](
  module_=NNClassifier(
    (Linear1): Linear(in_features=8, out_features=256, bias=True)
    (Linear2): Linear(in_features=256, out_features=256, bias=True)
    (Linear3): Linear(in_features=256, out_features=1, bias=True)
    (act1): ReLU()
    (act2): ReLU()
    (Dropout1): Dropout(p=0.1, in...
                   param_distributions={'batch_size': [8, 16],
                                        'callbacks': [[('EarlyStopping',
                                                        <skorch.callbacks.training.EarlyStopping object at 0x00000197FA63A610>)],
                                                      [('EarlyStopping',
                                                        <skorch.callbacks.training.EarlyStopping object at 0x00000197FA63A640>)]],
                                        'module__hiddenSize': [8, 16, 64, 128],
                             

In [13]:
rscv.best_params_

{'optimizer__weight_decay': 0,
 'module__p': 0.2,
 'module__hiddenSize': 16,
 'callbacks': [('EarlyStopping',
   <skorch.callbacks.training.EarlyStopping at 0x197fa63a640>)],
 'batch_size': 8}

In [14]:
# prediction for test data
y_pred = rscv.predict(X_test)
print(classification_report(y_test.squeeze(),y_pred))

              precision    recall  f1-score   support

           0       0.82      0.90      0.86       130
           1       0.74      0.60      0.66        62

    accuracy                           0.80       192
   macro avg       0.78      0.75      0.76       192
weighted avg       0.80      0.80      0.80       192

